In [ ]:
import os

import astropy.units as u
import numpy as np
import requests
from astropy.cosmology import Planck18 as cosmo
from astropy.table import Table
from matplotlib import pyplot as plt
from scipy.stats import ks_2samp

In [ ]:
def download_file(url, filename):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return filename
    except Exception as e:
        print(f"Error downloading file: {e}")
        return None

## 1. Read distribution sample.

In [ ]:
filename = "O4_result/baseline5_widesigmachi2_mass_NotchFilterBinnedPairingMassDistribution_redshift_powerlaw_mag_iid_spin_magnitude_gaussian_tilt_iid_spin_orientation_result_events_baseline5_all.h5"

In [ ]:
table = Table.read(filename)
table[:2]

## 2. Convert GWTC-4's distribution to suitable format for bayestar-inject

In [ ]:
z = table["redshift"]
distance = cosmo.luminosity_distance(z).to_value(u.Mpc)

gwtc4_samples = Table(
    {
        "mass1": table["mass_1"],
        "mass2": table["mass_2"],
        "spin1z": table["a_1"] * table["cos_tilt_1"],
        "spin2z": table["a_2"] * table["cos_tilt_2"],
        "distance": distance,
    }
)

## 3. Number of events in each sub-population

In [ ]:
ns_max_mass = 3
source_mass1 = gwtc4_samples["mass1"]
source_mass2 = gwtc4_samples["mass2"]

# classify systems

BNS = np.sum((source_mass1 < ns_max_mass) & (source_mass2 < ns_max_mass))
NSBH = np.sum((source_mass1 >= ns_max_mass) & (source_mass2 < ns_max_mass))
BBH = np.sum((source_mass1 >= ns_max_mass) & (source_mass2 >= ns_max_mass))

In [ ]:
# print results
print("CBC classification results:")
print(f"  BNS  : {BNS}")
print(f"  NSBH : {NSBH}")
print(f"  BBH  : {BBH}")

## GWTC-3 distribustion

In [ ]:
# gwtc3_samples = Table.read("./data/farah.h5")

# gwtc3_samples[:2]

file_url = "https://dcc.ligo.org/LIGO-T2100512/public/O1O2O3all_mass_h_iid_mag_iid_tilt_powerlaw_redshift_maxP_events_all.h5"
file_name = os.path.join("data", file_url.split("/")[-1])
input_file = download_file(file_url, file_name)

In [ ]:
gwtc3_table = Table.read(input_file)

z = gwtc3_table["redshift"]
distance = cosmo.luminosity_distance(z).to_value(u.Mpc)

gwtc3_samples = Table(
    {
        "mass1": gwtc3_table["mass_1"],
        "mass2": gwtc3_table["mass_2"],
        "spin1z": gwtc3_table["a_1"] * gwtc3_table["cos_tilt_1"],
        "spin2z": gwtc3_table["a_2"] * gwtc3_table["cos_tilt_2"],
        "distance": distance,
    }
)

In [ ]:
gwtc3_table[:2]

In [ ]:
ns_max_mass = 3
source_mass1 = gwtc3_samples["mass1"]
source_mass2 = gwtc3_samples["mass2"]

# classify systems

BNS = np.sum((source_mass1 < ns_max_mass) & (source_mass2 < ns_max_mass))
NSBH = np.sum((source_mass1 >= ns_max_mass) & (source_mass2 < ns_max_mass))
BBH = np.sum((source_mass1 >= ns_max_mass) & (source_mass2 >= ns_max_mass))

# print results
print("CBC classification results:")
print(f"  BNS  : {BNS}")
print(f"  NSBH : {NSBH}")
print(f"  BBH  : {BBH}")

# KS Test

In [ ]:
gwtc3_samples["log10_distance"] = np.log10(gwtc3_samples["distance"])
gwtc4_samples["log10_distance"] = np.log10(gwtc4_samples["distance"])

In [ ]:
for col in ["mass1", "mass2", "spin1z", "spin2z", "log10_distance"]:
    fig, ax = plt.subplots()
    ax.hist(
        [gwtc3_samples[col], gwtc4_samples[col]],
        label=["GWTC-3", "GWTC-4"],
        histtype="step",
        density=1,
        bins=50,
    )
    ax.set_ylim(0, None)
    ax.set_ylabel("pdf")
    ax2 = ax.twinx()
    ax2.set_ylabel("cdf")
    ax2.set_ylim(0, 1)
    ax2.plot(
        np.sort(gwtc3_samples[col]),
        np.linspace(0, 1, len(gwtc3_samples)),
        label="GWTC-3",
    )
    ax2.plot(
        np.sort(gwtc4_samples[col]),
        np.linspace(0, 1, len(gwtc4_samples)),
        label="GWTC-4",
    )
    ax.legend(loc="lower right")
    ax2.legend(loc="upper right")
    stat, pvalue = ks_2samp(gwtc4_samples[col], gwtc4_samples[col])
    ax.set_xlabel(col)
    ax.set_title(f"K-S test statistic={stat:0.3f}, P-value={pvalue:0.3g}")
    fig.savefig(f"ks_{col}.png", dpi=300)